In [13]:
import networkx as nx
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
import common as cmn
import itertools, traceback, copy, time, sys, os
import collisions as clsn

In [14]:
a_star_heuristic = lambda a,b : np.linalg.norm(np.array(a)-np.array(b))
pe = clsn.PolygonEnvironment()

In [18]:
startTime = time.time()

rrng = [0,40]
crng = [0,40]

maxDigs = 3

occupancyGrid = np.zeros(shape=(rrng[1],crng[1]),dtype=np.uint)

obsProb = .40

print 'generating occupancy grid...'

pickedStart = False
for r in range(rrng[1]):
    for c in range(crng[1]):
        rnd = np.random.uniform(0,1)
        if rnd < obsProb:
            occupancyGrid[(r,c)] = 2 # ONLY OBS
    

freeTmp = np.argwhere(occupancyGrid==0)
while True:
    try:
        # Add the start
        startIdx = freeTmp[np.random.choice(range(len(freeTmp)))]
        occupancyGrid[(startIdx[0],startIdx[1])] = 3
        break
    except:
        pass

freeTmp = np.argwhere(occupancyGrid==0)
digCnt = 0
while digCnt < maxDigs:
    try:
        # Add the start
        digIdx = freeTmp[np.random.choice(range(len(freeTmp)))]
        if occupancyGrid[(digIdx[0],digIdx[1])] == 0:
            occupancyGrid[(digIdx[0],digIdx[1])] = 1
            digCnt += 1
    except:
        pass


while True:
    try:
        # Add the start
        dumpIdx = freeTmp[np.random.choice(range(len(freeTmp)))]
        if occupancyGrid[(dumpIdx[0],dumpIdx[1])] == 0:
            occupancyGrid[(dumpIdx[0],dumpIdx[1])] = 4
            break
    except:
        pass

types = {}
obsNodes = []
freeNodes = []
digNodes = []
dumpNodes = []
startNodes = []
for r in range(rrng[1]):
    for c in range(crng[1]):
        types.update({(r,c):occupancyGrid[(r,c)]})
        if occupancyGrid[(r,c)] == 2:        #OBS
            obsNodes.append((r,c))
        elif occupancyGrid[(r,c)] == 1:      #DIG
            digNodes.append((r,c))
            freeNodes.append((r,c)) 
        elif occupancyGrid[(r,c)] == 3:      #START   
            startNodes.append((r,c))        
            freeNodes.append((r,c)) 
        elif occupancyGrid[(r,c)] == 4:
            dumpNodes.append((r,c))
            freeNodes.append((r,c))            
        else:                                #FREE
            freeNodes.append((r,c))            

print 'generating base nx grid...'
BaseGrid = nx.grid_2d_graph(rrng[1],crng[1])
r_len = 1.5
nx.set_edge_attributes(BaseGrid,r_len,'r')
nx.set_node_attributes(BaseGrid,types,'type') # 0: free   1: dig    2: obstacle   3: start

nodeSizes = []
nodeColors = []
edgeColors = []
edgeWidths = []
for node in BaseGrid.nodes(data=True):
    if node[1]['type'] == 0: # FREE SPACE
        nodeSizes.append(.1)
        nodeColors.append('k')
    if node[1]['type'] == 1: # DIG
        nodeSizes.append(70)
        nodeColors.append('g')
    if node[1]['type'] == 2: # OBS
        nodeSizes.append(50)
        nodeColors.append('r')
    if node[1]['type'] == 3: # START
        nodeSizes.append(70)
        nodeColors.append('c')
    if node[1]['type'] == 4: # Dump
        nodeSizes.append(70)
        nodeColors.append('m')


print '\tadding diags...'        

diagActions = ['ne','se','sw','nw']
diagEdges = []
# ADD DIAG EDGES
for r in range(rrng[1]):
    for c in range(crng[1]):
        for diag in diagActions:
            ((r2,c1),(r1,c2)) = cmn.getRCAfterAction(r,c,diag) # SANS ROW FLIP IN CMN
            if r2 >= 0 and r2<rrng[1] and c2 >= 0 and c2<crng[1]:
                diagEdges.append(((r,c),(r2,c2)))
BaseGrid.add_edges_from(diagEdges,r=np.sqrt(2)*r_len)

rmvEdgs = []
for edg in BaseGrid.edges(data=True):
    if (edg[0] in freeNodes and edg[1] in obsNodes) or \
       (edg[1] in freeNodes and edg[0] in obsNodes) or \
       (edg[0] in digNodes and edg[1] in obsNodes) or \
       (edg[1] in digNodes and edg[0] in obsNodes) or \
       (edg[0] in startNodes and edg[1] in obsNodes) or \
       (edg[1] in startNodes and edg[0] in obsNodes):
        rmvEdgs.append(edg)
BaseGrid.remove_edges_from(rmvEdgs)

#print obsNodes

print 'generating obstacle nx grid...'
obsGrid  = BaseGrid.subgraph(obsNodes).copy()

#print obsGrid.edges()

vrs = 'orig' # ORIG IS FASTER
if vrs == 'orig':
    rmvEdgs = []
    ndx = 0
    obsSize = len(list(obsGrid.edges()))
    edgeCheckSubStart = time.time()
    for edg in obsGrid.edges(data=True):
        if ndx%(int(np.ceil(obsSize*.01)))==0:
            print '\t\t{} of {} obs edges checked in {} S'.format(ndx,obsSize,time.time()-edgeCheckSubStart)
            edgeCheckSubStart = time.time()
        for edgBase in BaseGrid.edges(data=True):
            if not edgBase in obsGrid.edges(data=True):
                if pe.line_line_collision(edg,edgBase):
                    rmvEdgs.append(edgBase)
        ndx+=1
    BaseGrid.remove_edges_from(rmvEdgs)
elif vrs == 'new':
    rmvEdgs = []
    ndx = 0
    obsSize = len(list(obsGrid.edges()))
    edgeCheckSubStart = time.time()
    for edg in obsGrid.edges(data=True):
        if ndx%(int(np.ceil(obsSize*.01)))==0:
            print '\t\t{} of {} obs edges checked in {} S'.format(ndx,obsSize,time.time()-edgeCheckSubStart)
            edgeCheckSubStart = time.time()

        #np.any([(edg[0]==(r2,c2) or edg[1]==(r2,c2)) for ((r1,c1),(r2,c2)) in [cmn.getRCAfterAction(edg[0][0], edg[0][1], tmpAct) for tmpAct in cmn._actions]])
        #for edgBase in [tmpedg for tmpedg in self.baseGrid.edges(data=True) if:

        for edgBase in [tmpedg for tmpedg in BaseGrid.edges(data=True) if np.any([(edg[0]==(r2,c2) or edg[1]==(r2,c2)) for ((_,_),(r2,c2)) in [cmn.getRCAfterAction(edg[0][0], edg[0][1], tmpAct) for tmpAct in cmn._actions]])]:
            if not edgBase in obsGrid.edges(data=True):
                if pe.line_line_collision(edg, edgBase):
                    rmvEdgs.append(edgBase)
        ndx += 1
    BaseGrid.remove_edges_from(rmvEdgs)

for edg in BaseGrid.edges(data=True):
    if edg[0] in obsNodes or edg[1] in obsNodes:
        edgeColors.append('b')
        edgeWidths.append(2)
    else:
        edgeColors.append('dimgrey')
        edgeWidths.append(.1)

print 'saving initial base grid...'
nx.draw_networkx(BaseGrid,pos={k:np.array(k) for (k,v) in BaseGrid.nodes(data=True)},node_size=nodeSizes, node_color=nodeColors, edge_color=edgeColors, linewidths=edgeWidths, with_labels=False)
plt.xlim((rrng[0]-1,rrng[1]))
plt.ylim((crng[0]-1,crng[1]))
fig = plt.gcf()
rc_ratio = float(rrng[1])/crng[1]
fig.set_size_inches(11,int(np.ceil(11*rc_ratio)))
ax = plt.gca()
ax.set_facecolor('ghostwhite')
bbox_props = dict(boxstyle="square", fc="w", ec="0.7", alpha=0.5)
for digIdx in range(len(digNodes)):
    tmpTxtPos = list(digNodes[digIdx])
    if rrng[1] < 15: tmpTxtPos[1] -= .8
    else:            tmpTxtPos[1] -= 1.5
    ax.text(tmpTxtPos[0], tmpTxtPos[1], 'Dig'.format(digIdx+1), ha="center", va="center", size=20,
        bbox=bbox_props)
for digIdx in range(len(startNodes)):
    tmpTxtPos = list(startNodes[digIdx])
    if rrng[1] < 15: tmpTxtPos[1] -= .8
    else:            tmpTxtPos[1] -= 1.5
    ax.text(tmpTxtPos[0], tmpTxtPos[1], 'Start', ha="center", va="center", size=20,
        bbox=bbox_props)
for digIdx in range(len(dumpNodes)):
    tmpTxtPos = list(dumpNodes[digIdx])
    if rrng[1] < 15: tmpTxtPos[1] -= .8
    else:            tmpTxtPos[1] -= 1.5
    ax.text(tmpTxtPos[0], tmpTxtPos[1], 'Dump', ha="center", va="center", size=20,
        bbox=bbox_props)
plt.savefig(os.path.join('imgs','dig_site_ordering_r{}_c{}_op{}_d{}_INIT.svg'.format(rrng[1],crng[1],int(np.ceil(obsProb*100.0)),maxDigs)))
plt.savefig(os.path.join('imgs','dig_site_ordering_r{}_c{}_op{}_d{}_INIT.png'.format(rrng[1],crng[1],int(np.ceil(obsProb*100.0)),maxDigs)))
plt.draw()
plt.cla()
plt.clf()
plt.close()

print 'generating free nx grid...'
freeGrid = BaseGrid.subgraph(freeNodes).copy()
digGrid = BaseGrid.subgraph(digNodes).copy()
print 'grid generation took {} s...\n'.format(time.time()-startTime)

digOrderings = list(itertools.permutations(digNodes))
shortestRoverPathNodes = None
shortestRoverPathEdges = None
shortestRoverPathLength = 0.0
pathLengths = []
shortestDigOrder = []
for digOrderIdx in range(len(digOrderings)):
    digOrder = digOrderings[digOrderIdx]
    #print digOrder
    print '\nChecking out combo {} of {}'.format(digOrderIdx+1,len(digOrderings))
    startPos = startNodes[0]
    currPos = startPos
    dumpPos = dumpNodes[0]
    roverPathNodes = []
    roverPathEdges = []
    MutatedBaseGrid = BaseGrid.copy()
    totalLength = 0.0
    print '\tCalculating dig ',; sys.stdout.flush()
    broken = False
    for digIdx in range(len(digOrder)):
        print '{} at {}..  '.format(digIdx+1,digOrder[digIdx]),; sys.stdout.flush()
        try:
            tmpPath = nx.astar_path(MutatedBaseGrid,
                              source=currPos,
                              target=digOrder[digIdx],
                              heuristic=a_star_heuristic,
                              weight='r')
            edgs = list(zip(tmpPath,tmpPath[1:]))
            tmpLen = 0.0
            for edg in edgs:
                try:
                    tmpLen += nx.get_edge_attributes(MutatedBaseGrid, 'r')[edg]
                except:
                    pass
            roverPathNodes.append(tmpPath)
            roverPathEdges.append(edgs)
            totalLength += tmpLen
            currPos = digOrder[digIdx]
        except:
            #print '\t\tfailed for dig {} on {} --> {}\n{}\n'.format(digIdx+1,currPos,digOrder[digIdx],traceback.format_exc())
            broken = True
            break
        try:
            tmpPath = nx.astar_path(MutatedBaseGrid,
                              source=currPos,
                              target=dumpPos,
                              heuristic=a_star_heuristic,
                              weight='r')
            edgs = list(zip(tmpPath,tmpPath[1:]))
            tmpLen = 0.0
            for edg in edgs:
                try:
                    tmpLen += nx.get_edge_attributes(MutatedBaseGrid, 'r')[edg]
                except:
                    pass
            roverPathNodes.append(tmpPath)
            roverPathEdges.append(edgs)
            totalLength += tmpLen
            currPos = dumpPos
        except:
            #print '\t\tfailed for dump {} on {} --> {}\n{}\n'.format(digIdx+1,currPos,dumpPos,traceback.format_exc())
            broken = True
            break
        MutatedBaseGrid.remove_node(digOrder[digIdx])
    if broken: continue
    pathLengths.append(totalLength)
    if shortestRoverPathNodes is None:
        shortestRoverPathNodes = copy.copy(roverPathNodes)
        shortestRoverPathEdges = copy.copy(roverPathEdges)
        shortestDigOrder = digOrder
        shortestRoverPathLength = totalLength
    else:
        if totalLength < shortestRoverPathLength:
            shortestRoverPathNodes = copy.copy(roverPathNodes)
            shortestRoverPathEdges = copy.copy(roverPathEdges)
            shortestDigOrder = digOrder
            shortestRoverPathLength = totalLength

print '\nPATH LENGTH STATS\n\n{}\n\n'.format(stats.describe(pathLengths))

print 'plotting total path of {}...'.format(shortestRoverPathLength)

nx.draw_networkx(BaseGrid,pos={k:np.array(k) for (k,v) in BaseGrid.nodes(data=True)},node_size=nodeSizes, node_color=nodeColors, edge_color=edgeColors, linewidths=edgeWidths, with_labels=False)
#nx.draw_networkx_nodes(BaseGrid,pos={k:np.array(k) for (k,v) in BaseGrid.nodes(data=True)},nodelist=digPathNodes[digToPlotIdx], node_color='y', node_size=75)
#nx.draw_networkx_edges(BaseGrid,pos={k:np.array(k) for (k,v) in BaseGrid.nodes(data=True)},edgelist=digPathEdges[digToPlotIdx], edge_color='y', width=1)

clrmp = 'Blues'

for i in range(len(shortestRoverPathNodes)):
    nx.draw_networkx_nodes(BaseGrid,pos={k:np.array(k) for (k,v) in BaseGrid.nodes(data=True)},nodelist=shortestRoverPathNodes[i], node_color=[i]*len(shortestRoverPathNodes[i]), vmin=-5, vmax=len(shortestRoverPathNodes)+1, cmap=clrmp, node_size=130)
    nx.draw_networkx_edges(BaseGrid,pos={k:np.array(k) for (k,v) in BaseGrid.nodes(data=True)},edgelist=shortestRoverPathEdges[i], width=1)# edge_color =[i]*len(shortestRoverPathEdges[i]), edge_vmin=-3, edge_vmax=len(shortestRoverPathNodes), edgecmap=clrmp, width=2)

nx.draw_networkx_nodes(BaseGrid,pos={k:np.array(k) for (k,v) in BaseGrid.nodes(data=True)},nodelist=digNodes, node_color='seagreen', node_size=100)
nx.draw_networkx_nodes(BaseGrid,pos={k:np.array(k) for (k,v) in BaseGrid.nodes(data=True)},nodelist=startNodes, node_color='mediumvioletred', node_size=200)
nx.draw_networkx_nodes(BaseGrid,pos={k:np.array(k) for (k,v) in BaseGrid.nodes(data=True)},nodelist=dumpNodes, node_color='gold', node_size=200)

#nx.draw_networkx_nodes(BaseGrid,pos={k:np.array(k) for (k,v) in BaseGrid.nodes(data=True)},nodelist=digGrid.nodes(), node_color='g', node_size=75)
#nx.draw_networkx_edges(BaseGrid,pos={k:np.array(k) for (k,v) in BaseGrid.nodes(data=True)},edgelist=digGrid.edges(), edge_color='g', width=1)

plt.xlim((rrng[0]-1,rrng[1]))
plt.ylim((crng[0]-1,crng[1]))
fig = plt.gcf()
rc_ratio = float(rrng[1])/crng[1]
fig.set_size_inches(11,int(np.ceil(11*rc_ratio)))
ax = plt.gca()
ax.set_facecolor('ghostwhite')

bbox_props = dict(boxstyle="square", fc="w", ec="0.7", alpha=0.5)
for digIdx in range(len(shortestDigOrder)):
    tmpTxtPos = list(shortestDigOrder[digIdx])
    
    if rrng[1] < 15: tmpTxtPos[1] -= .8
    else:            tmpTxtPos[1] -= 1.5
        
    ax.text(tmpTxtPos[0], tmpTxtPos[1], 'Dig {}'.format(digIdx+1), ha="center", va="center", size=20,
        bbox=bbox_props)
for digIdx in range(len(startNodes)):
    tmpTxtPos = list(startNodes[digIdx])
    if rrng[1] < 15: tmpTxtPos[1] -= .8
    else:            tmpTxtPos[1] -= 1.5
    ax.text(tmpTxtPos[0], tmpTxtPos[1], 'Start', ha="center", va="center", size=20,
        bbox=bbox_props)
for digIdx in range(len(dumpNodes)):
    tmpTxtPos = list(dumpNodes[digIdx])
    if rrng[1] < 15: tmpTxtPos[1] -= .8
    else:            tmpTxtPos[1] -= 1.5
    ax.text(tmpTxtPos[0], tmpTxtPos[1], 'Dump', ha="center", va="center", size=20,
        bbox=bbox_props)
                

totTime = time.time()-startTime
print 'total execution time: {} s'.format(totTime)
plt.savefig(os.path.join('imgs','dig_site_ordering_r{}_c{}_op{}_d{}_t{}.svg'.format(rrng[1],crng[1],int(np.ceil(obsProb*100.0)),maxDigs,int(np.ceil(totTime)))))
plt.savefig(os.path.join('imgs','dig_site_ordering_r{}_c{}_op{}_d{}_t{}.png'.format(rrng[1],crng[1],int(np.ceil(obsProb*100.0)),maxDigs,int(np.ceil(totTime)))))
plt.draw()

 generating occupancy grid...
generating base nx grid...
	adding diags...
generating obstacle nx grid...
		0 of 1109 obs edges checked in 0.0 S
		12 of 1109 obs edges checked in 0.802999973297 S
		24 of 1109 obs edges checked in 0.84299993515 S
		36 of 1109 obs edges checked in 0.680999994278 S
		48 of 1109 obs edges checked in 0.695999860764 S
		60 of 1109 obs edges checked in 0.563999891281 S
		72 of 1109 obs edges checked in 0.559000015259 S
		84 of 1109 obs edges checked in 0.582999944687 S
		96 of 1109 obs edges checked in 0.611999988556 S
		108 of 1109 obs edges checked in 0.733999967575 S
		120 of 1109 obs edges checked in 0.736000061035 S
		132 of 1109 obs edges checked in 0.668999910355 S


KeyboardInterrupt: 